## Orange Team CQ#1.7

### Query: 
What genes show high phenotypic similarity to the 11 Fanconi Anemia core complex genes (set FA-core)?

### Services:
BioLink API (Monarch) - https://api.monarchinitiative.org/api/
Owlsim - http://owlsim3.monarchinitiative.org/api/docs/

### Approach:
As a start we will get all human gene - human gene sim scores, this notebook can be adjusted
to search for model organism genes instead

### Input:
Hardcoded tsv file from: https://raw.githubusercontent.com/NCATS-Tangerine/cq-notebooks/master/OrangeQ1.5_Regulatory_Motif_Signature/FA_NCBIGene_symbol.txt

In [6]:
import biolink_client
from biolink_client.api_client import ApiClient
from biolink_client.rest import ApiException
import requests
import pandas as pd
from pprint import pprint

MONARCH_API = "https://api.monarchinitiative.org/api"
OWLSIM_API = "http://owlsim3.monarchinitiative.org/api"

gene_list = "https://raw.githubusercontent.com/NCATS-Tangerine/cq-notebooks/" \
            "master/OrangeQ1.5_Regulatory_Motif_Signature/FA_NCBIGene_symbol.txt"

client = ApiClient(host=MONARCH_API)
client.set_default_header('Content-Type', 'text/plain')
api_instance = biolink_client.BioentityApi(client)

# Get the gene list from github
dataframe = pd.read_csv(gene_list, sep='\t', names=['gene_id', 'symbol'])
df = dataframe.set_index('symbol')
gene_hpo_map = dict()

for index, row in df.iterrows():
    api_response = api_instance.get_gene_phenotype_associations(row['gene_id'], rows=500)
    # TODO add facet_counts to AssociationResults model
    # TODO use facet_counts to check the gene does not have >500 phenotypes
    # TODO or better, add pagination
    gene_hpo_map[row['gene_id']] = api_response.objects

# Get the first five phenotypes for FANCA
pprint(gene_hpo_map[df.at['FANCA', 'gene_id']][0:5])

['EFO:0003924', 'EFO:0003963', 'HP:0000010', 'HP:0000027', 'HP:0000028']


In [7]:
# Search for top human genes
# TODO implement prefix or taxon+type filters in owlsim
# TODO fix cutoff filter

# Use phenodigm algorithm, with a cutoff of 70/100
matcher = 'phenodigm'
score_cutoff = 70

result_set = []


for index, row in df.iterrows():
    params = {
        'id': gene_hpo_map[row['gene_id']]
    }
    url = "{}/match/{}".format(OWLSIM_API, matcher)
    req = requests.get(url, params=params)
    owlsim_results = req.json()
    for match in owlsim_results['matches']:
        try:
            if match['matchId'].startswith("NCBIGene")\
                    and match['rawScore'] >= score_cutoff:
                result = [row['gene_id'],index, match['matchId'],
                          match['matchLabel'], match['rawScore']]
                result_set.append(result)
        except TypeError as e:
            # TypeError when score is NaN
            #print(e)
            #print(match)
            continue


# Create a table of query gene, matched gene, and sim score
column_names = ['query_gene', 'query_symbol', 'match_gene', 'match_symbol', 'sim_score']
result_frame = pd.DataFrame(data=result_set, columns=column_names)

# Get sim scores for FANCA
df = result_frame.loc[result_frame['query_symbol'] == 'FANCA']

print(df.head(30))

       query_gene query_symbol      match_gene match_symbol   sim_score
0   NCBIGene:2175        FANCA   NCBIGene:2175        FANCA  100.000000
1   NCBIGene:2175        FANCA   NCBIGene:2176        FANCC   98.139284
2   NCBIGene:2175        FANCA   NCBIGene:2177       FANCD2   98.139284
3   NCBIGene:2175        FANCA   NCBIGene:2178        FANCE   98.139284
4   NCBIGene:2175        FANCA  NCBIGene:55215        FANCI   90.985447
5   NCBIGene:2175        FANCA  NCBIGene:83990        BRIP1   90.690054
6   NCBIGene:2175        FANCA  NCBIGene:10459       MAD2L2   89.852884
7   NCBIGene:2175        FANCA  NCBIGene:57697        FANCM   89.852884
8   NCBIGene:2175        FANCA   NCBIGene:2188        FANCF   89.852884
9   NCBIGene:2175        FANCA   NCBIGene:2189        FANCG   89.852884
10  NCBIGene:2175        FANCA   NCBIGene:7516        XRCC2   89.852884
11  NCBIGene:2175        FANCA  NCBIGene:84464         SLX4   89.785415
12  NCBIGene:2175        FANCA   NCBIGene:5888        RAD51   89

In [8]:
# Get sim scores for ERCC4
df = result_frame.loc[result_frame['query_symbol'] == 'ERCC4']

print(df.head(30))

        query_gene query_symbol      match_gene match_symbol   sim_score
320  NCBIGene:2072        ERCC4   NCBIGene:2072        ERCC4  100.000000
321  NCBIGene:2072        ERCC4  NCBIGene:10459       MAD2L2   81.281953
322  NCBIGene:2072        ERCC4  NCBIGene:57697        FANCM   81.281953
323  NCBIGene:2072        ERCC4   NCBIGene:2188        FANCF   81.281953
324  NCBIGene:2072        ERCC4   NCBIGene:2189        FANCG   81.281953
325  NCBIGene:2072        ERCC4   NCBIGene:7516        XRCC2   81.281953
326  NCBIGene:2072        ERCC4   NCBIGene:5888        RAD51   81.181011
327  NCBIGene:2072        ERCC4  NCBIGene:29089        UBE2T   81.181011
328  NCBIGene:2072        ERCC4  NCBIGene:55215        FANCI   80.980745
329  NCBIGene:2072        ERCC4  NCBIGene:55120        FANCL   80.927818
330  NCBIGene:2072        ERCC4   NCBIGene:5889       RAD51C   80.918673
331  NCBIGene:2072        ERCC4  NCBIGene:83990        BRIP1   80.911740
332  NCBIGene:2072        ERCC4  NCBIGene:84464    

In [13]:
# Many genes are from our original list
# Filter out all genes from the input set

filtered_frame = result_frame[~result_frame['match_gene'].isin(gene_hpo_map.keys())]

# Get sim scores for ERCC4
df = filtered_frame.loc[filtered_frame['query_symbol'] == 'ERCC4']

print(df.head(30))

        query_gene query_symbol     match_gene match_symbol  sim_score
339  NCBIGene:2072        ERCC4  NCBIGene:2073        ERCC5  77.792617
340  NCBIGene:2072        ERCC4  NCBIGene:2071        ERCC3  77.364768
342  NCBIGene:2072        ERCC4  NCBIGene:2068        ERCC2  75.551625
343  NCBIGene:2072        ERCC4  NCBIGene:1643         DDB2  74.510667
344  NCBIGene:2072        ERCC4  NCBIGene:7508          XPC  74.098468
345  NCBIGene:2072        ERCC4  NCBIGene:7507          XPA  73.973250
